In [11]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import date
from pandas_datareader.data import DataReader

from hmmlearn.hmm import GaussianHMM

In [9]:
start = date(1993, 2, 8)
spy = DataReader('SPY', 'yahoo', start=start)

In [7]:
spy_ret = spy
spy_ret = spy_ret.reset_index()
spy_ret["Returns"] = spy_ret['Adj Close'].pct_change()
spy_ret.dropna(inplace=True)
spy_ret.to_csv('spy_ret.csv')

In [ ]:
if __name__=="__main__":
    #Hides deprecation warnings for sklearn
    warnings.filterwarnings("ignore")
    
    #Create the SPY dataframe from the Yahoo Finance CSV
    # and correctly format the returns for use in the HMM
    csv_filepath = "..SPY.csv"
    pickle_path = "../Users/bng19/Documents/nss_projects/Bayes_will_conquer_all"
    end_date = datetime.datetime(2005, 12, 31)
    spy = obtain_prices_df(csv_filepath, end_date)
    rets = np.column_stack([spy["Returns"]])
    
    #create the Gaussian Hidden markov Model and fit it
    #to the SPY returns, dta, outputting a score
    hmm_model = GaussianHMM(
        n_components=2, 
        covariance_type = "full",
        n_iter=1000
    ).fit(rets)
    print("Model Score:", hmm_model.score(rets))
    
    #Plot the in sample hidden states closing values
    plot_in_sample_hidden_states(hmm_model, spy)
    
    print("Pickling HMM model...")
    pickle.dump(hmm_model, open(pickle_path, "wb"))
    print("...HMM model pickled")

In [11]:
#borrow from advanced algo trading to plot in sample hidden states
def plot_in_sample_hidden_states(hmm_model, df):
    '''
    Plot the adjusted closing prcies masked by 
    the in-sample hidden states as a mechanism 
    to understand the market regimes.
    '''
    #Predict the hidden sstates array
    hidden_states = hmm_model.predict(rets)
    # Create the correctly formatted plot
    fig, axs = plt.subplots(
    hmm_nodel.n_components,
    sharex=True, sharey=True
    )
    colors = cm.rainbow(
    np.linspace(0, 1, hmm_model.n_components)
    )
    for i, (ax, color)in enumerate(zip(axs, colors)):
        mask = hidden_states == i
        ax.plot_date(
            spy_ret.index[mask],
            spy_ret['Adj Close'][mask],
            ".", linstyle='none',
            c=color
        )
        ax.set_title("Hiddden State #%s" % i)
        ax.xaxis.set_major_locator(YearLocator())
        ax.xaxis.set_minor_locator(MonthLocator())
        ax.grid(True)
    plt.show()
    
